In [ ]:
#install all the dependencies
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("/content/file.csv")
df

In [ ]:
df.shape
df.columns
df["Outcome"].value_counts()
sns.countplot(x="Outcome",data=df)

In [ ]:
#use for autoanalysis
!pip install sweetviz

In [ ]:
import sweetviz as sv
report=sv.analyze(df)
report.show_html("eda_report_15.html")

In [ ]:
#divide x and y
x=df.drop(columns=["Outcome"],axis=1)
y=df["Outcome"]

In [ ]:
#train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [ ]:
#whenever we are working on ML or DL we need to scale the data
#in case of DL scaling is required we should do always scaling.
scaler=StandardScaler()
x_train_scaled=pd.DataFrame(scaler.fit_transform(x_train),columns=x_train.columns)
x_test_scaled=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [ ]:
#building the model
model=tf.keras.Sequential(name="Main_container")
#sequntial=linearly stats layer
#in DL two types of models were build 1st sequntial and 2nd functional
#functional are complex model(multiple output and multiple input)
#name= we can give any name
model.add(tf.keras.layers.Input(shape=[8],name="input_layer"))
#shape means how much input columns ( 8 layers)
#there are no calculations in input layers
model.add(tf.keras.layers.Dense(units=9,activation="elu",kernel_initializer="he_normal",name="hidden_layer_1"))
#Dense input means here processing of inputs happens
#units means number of neurons(randomly)
#activation means activation function
#kernel_initializer=weights associated hotat tyasathi char techniques astat like he normal, he_uniform, glorot_normal, glorot_uniform
model.add(tf.keras.layers.Dense(units=8,activation="elu",kernel_initializer="he_normal",name="hidden_layer_2"))
model.add(tf.keras.layers.Dense(units=5,activation="elu",kernel_initializer="he_normal",name="hidden_layer_3"))
#output
model.add(tf.keras.layers.Dense(units=1,activation="sigmoid",kernel_initializer="he_normal",name="output_layer"))
#model architecture complete

In [ ]:
model.summary()
#params means total weights and biases like 9*8=72+81
#or 8*8=70+8=80

In [ ]:
model.layers[0].get_weights()
#here we get idea 

In [ ]:
plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    show_layer_activations=True
)
#here it shows architecture

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
!pip install keras
!pip install ann_visualizer
!pip install graphviz

In [ ]:
from ann_visualizer.visualize import ann_viz

ann_viz(model

In [ ]:
history=model.fit(
    x_train_scaled,
    y_train,
    verbose=True,
    batch_size=16,
    validation_split=0.2,
    epochs=50,
    workers=10,
    use_multiprocessing=True
)
#verbose=model je pn backend la work krnr te sangt rahnar
#batch_size=depend on dataset size
#validation_split=
#workers=thread kiti asavet
#use_multiprocessing=its impactful when we have bigger dataset

In [ ]:
history.history
#same data in dict form, whenever we want to plt data its use

In [ ]:
def plotting(history, keyword):
  plt.plot(history.history[keyword])
  plt.plot(history.history["val_" + keyword])

  plt.title("model" + keyword)
  plt.ylabel(keyword)
  plt.xlabel('epoch')

  plt.legend(["training data","validation data"])

#ek accuracy training chi an ek validation chi soo used val_

In [ ]:
plotting(history,"accuracy")

In [ ]:
plotting(history,"loss")
#plotting loss graph

In [ ]:
y_logs=model.predict(x_test_scaled)
y_pred=np.where(y_logs>0.5,1,0)
#means o.5 peksha jast asel tr 1 nahitr 0
y_pred[:3]
#means mala starting che 3 show kra
y_logs

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test,y_pred)
acc

In [ ]:
#callbacks = tasks la automate krt mhnje 80% accuracy bhetlyavr stop kra

In [ ]:
class Conditionalcallback(tf.keras.callbacks.Callback):
    def end_epoch(self,epoch,logs={}):
        if (logs.get("accuracy")>=0.85 and logs.get("val_accuracy")>=0.85):
            self.model.stop_training=True
            #here we create call back to stop the model

In [ ]:
conditional_callback=Conditionalcallback

In [ ]:
early_stopper=EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.02,
    patience=16,
    mode="max",
    restore_best_weights=True
)

#min delta is 0.02 jr 2 percent peksha jast lvkr move hot nahi then stop
#te sangnar kon tr patience= 16 epoch prynt 2% peksha jast kami nahi zal tri stop
#restore best weights= kdhi as hou shkt last la chngla wt nsel so hyane adhicha changla result ghete

In [ ]:
input_data=(6,148,72,35,0,33.6,0.627,50)
input_data=np.asarray(input_data)
input_data_reshape=input_data.reshape(1,-1)
x_test_scaling=scaler.transform(input_data_reshape)
prediction=model.predict(x_test_scaling)
prediction
y_pred=np.where(prediction>0.5,1,0)
if y_pred[0][0]==1:
    print("person is suffering from diabetes".upper())
else:
    print("person is not suffering from diabetes".upper())

#here we predict the values

In [ ]:
import pickle
filename="trained_model.sav"
pickle.dump(model,open(filename,"wb"))

filename_sc="scaler.pkl"
pickle.dump(scaler,open(filename_sc,"wb"))

#create pickle file

In [ ]:
loaded_model=pickle.load(open("trained_model.sav","rb"))

loaded_scaler=pickle.load(open("scaler.pkl","rb"))
#create model which can predict

In [ ]:
input_data=(1,85,66,29,0,26.6,0.351,31)
input_data=np.asarray(input_data)
input_data_reshape=input_data.reshape(1,-1)
x_test_scaling=loaded_scaler.transform(input_data_reshape)
prediction=loaded_model.predict(x_test_scaling)
prediction
y_pred=np.where(prediction>0.5,1,0)
if y_pred[0][0]==1:
  print("person is suffering from diabetes".upper())
else:
  print("person is not suffering from diabetes".upper())
#predict with only loaded model
#means adhichi file delete zali tri run honar

In [ ]:
!pip install -q streamlit
#ek chota prediction website bnvun denar

In [ ]:
%%writefile app.py


import numpy as np
import pandas as pd
import streamlit as st
import pickle

loaded_model=pickle.load(open("/content/trained_model.sav","rb"))

loaded_scaler=pickle.load(open("/content/scaler.pkl","rb"))

def daibetes_prediction(input_data):
  input_data=np.asarray(input_data)
  input_data_reshape=input_data.reshape(1,-1)
  x_test_scaling=loaded_scaler.transform(input_data_reshape)
  prediction=loaded_model.predict(x_test_scaling)
  y_pred=np.where(prediction>0.5,1,0)
  if y_pred[0][0]==1:
    print("person is suffering from diabetes".upper())
  else:
    print("person is not suffering from diabetes".upper())

def main():
  st.title("Diabetes Prediction Web Application")
  Pregnancies = st.text_input("Pregnancies")
  Glucose=st.text_input("Glucose")
  BloodPressure=st.text_input("BloodPressure")
  SkinThickness=st.text_input("SkinThickness")
  Insulin=st.text_input("Insulin")
  BMI=st.text_input("BMI")
  DiabetesPedigreeFunction=st.text_input("DiabetesPedigreeFunction")
  Age=st.text_input("Age")

  diagnosis=""
  if st.button(Di8abetes Text Result):
    diagnosis = daibetes_prediction([Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age])
  st.success(diagnosis)

if __name__=="__main__":
  main()

#create page

In [ ]:
",".join(df.columns[:-1])

In [ ]:
!pip install pyngrok
pip install protobuf==3.20.*

#docker run -it python:3.10 /bin/bash
#within the Docker container:
!pip install google-cloud-logging==3.1.1 protobuf==4.21.0
!python3 -c "import google.cloud.logging"

!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

pip install --upgrade "protobuf<=3.20.1"

!pip install "protobuf<=3.20.1" --force-reinstall

!streamlit run app.py & npx localtunnel --port 8501